In [ ]:
import jax
import jax.numpy as jnp

x = jnp.array([1, 2, 3])

In [ ]:
array_device = x.device()
array_device

In [ ]:
import numpy as np
a = np.array([1, 2, 30])
a

In [ ]:
import torch
t = torch.from_numpy(a)
top_k=2
torch.topk(t, top_k, dim=-1)

In [ ]:
jax.lax.top_k(a, top_k)

In [ ]:
a.reshape(-1, 3)

In [ ]:
t = torch.from_numpy(a)
print(t)
t.reshape(-1, 3)

In [ ]:
jnp.mean(a)

In [ ]:
torch.mean(t.float())

In [ ]:
jnp.expand_dims(a, axis=1)

In [ ]:
t.expand(3)

In [ ]:
random_array = np.random.uniform(1, 10, (1, 100))
random_array.shape

In [ ]:
np.expand_dims(random_array, axis=0).shape

In [ ]:
torch.from_numpy(random_array).shape

In [ ]:
torch.from_numpy(random_array).unsqueeze(0).shape

In [ ]:
num_hidden_layers=12
batch_size=1
sequence_length=100
num_experts=2

expanded_shape = (num_hidden_layers, batch_size, sequence_length, 2, num_experts)
broadcasted_random_array = random_array[None, :, :, None, None]

# broadcasted_random_array = jnp.tile(broadcasted_random_array, expanded_shape)
broadcasted_random_array = jnp.broadcast_to(broadcasted_random_array, expanded_shape)

# # Proceeding with the similar operations as before
expert_attention_mask = jnp.reshape(broadcasted_random_array, (-1, 2, num_experts))
expert_attention_mask, expert_attention_mask.shape

In [ ]:
expert_attention_mask = (
            torch.from_numpy(random_array[None, :, :, None, None])
            .expand(expanded_shape)
            .reshape(-1, 2, num_experts)
        )
expert_attention_mask, expert_attention_mask.shape

In [ ]:
import torch

# Assuming x is a PyTorch tensor
x = torch.arange(16)  # Creates a tensor from 0 to 15

# Using view to change the shape of the tensor
y = x.view(4, 4)  # Reshapes x to a 4x4 tensor

print("PyTorch version:", y)

In [ ]:
import jax.numpy as jnp

# Assuming x is a JAX array
x = jnp.arange(16)  # Creates an array from 0 to 15

# Using reshape to change the shape of the array
x = x.reshape((4, 4))  # Reshapes x to a 4x4 array

print("Flax/JAX version:", x)


In [ ]:
batch_size = 2
sequence_length = 10
hidden_dim = 2
torch.zeros(
            (batch_size * sequence_length, hidden_dim)
        ).permute(2, 1, 0)

In [ ]:
jnp.zeros((batch_size * sequence_length, hidden_dim)).shape

In [ ]:
import torch

# Assuming x is a PyTorch tensor with shape [2, 3, 4]
x = torch.ones(2, 3, 4)
print(x.shape)
# Permute the dimensions of x
y = x.permute(2, 0, 1)  # Rearranges dimensions to [4, 2, 3]

print("PyTorch version, shape:", y.shape)
y.tolist()

In [ ]:
import jax.numpy as jnp

# Assuming x is a JAX array with shape [2, 3, 4]
x = jnp.ones((2, 3, 4))
print(x.shape)
# Transpose the dimensions of x
x = x.transpose((2, 0, 1))  # Rearranges dimensions to [4, 2, 3]

print("Flax/JAX version, shape:", x.shape)
x.tolist()


In [ ]:
x.reshape(4,2,3)

In [ ]:
from transformers import AutoTokenizer, MixtralModel

model = MixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)

In [5]:
from transformers import AutoTokenizer, FlaxMixtralModel

model = FlaxMixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

x = model(**inputs)

TypeError: __init__() got multiple values for argument 'i'

In [ ]:
from transformers import AutoTokenizer, FlaxLlamaModel

model = FlaxLlamaModel.from_pretrained("hf-internal-testing/tiny-random-LlamaForCausalLM")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

x = model(**inputs)